- 입력 : 광진구를 지나는 모든 버스의 노선별 승차 승객수
- 출력 : 광진구 내 버스 정류장별 승차 승객수

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./data/gwangjin_boarding.csv', encoding='cp949')

## 1. 시간대별로 나누기

In [19]:
# 출근시간 7 ~ 9
# 낮 시간 10 ~ 17
# 퇴근시간 18 ~ 20
# 밤시간 21 ~ 2

morning_cols = ['6시승차총승객수', '7시승차총승객수', '8시승차총승객수','9시승차총승객수']
afternoon_cols = ['10시승차총승객수', '11시승차총승객수', '12시승차총승객수', '13시승차총승객수', '14시승차총승객수', '15시승차총승객수', '16시승차총승객수', '17시승차총승객수']
evening_cols = ['18시승차총승객수', '19시승차총승객수', '20시승차총승객수', '21시승차총승객수']
night_cols = ['22시승차총승객수', '23시승차총승객수', '00시승차총승객수', '1시승차총승객수', '2시승차총승객수', '3시승차총승객수', '4시승차총승객수', '5시승차총승객수']

In [27]:
# 집계
data['출근시간대총승객수'] = data[morning_cols].sum(axis=1)
data['낮시간대총승객수'] = data[afternoon_cols].sum(axis=1)
data['퇴근시간대총승객수'] = data[evening_cols].sum(axis=1)
data['밤시간대총승객수'] = data[night_cols].sum(axis=1)

In [28]:
# 결과 확인
data.head()

,사용년월,표준버스정류장ID,버스정류장ARS번호,00시승차총승객수,1시승차총승객수,2시승차총승객수,3시승차총승객수,4시승차총승객수,5시승차총승객수,6시승차총승객수,...,위도,경도,버스도착정보안내기_설치_여부,구이름,행정동,일일총승객수,출근시간대총승객수,낮시간대총승객수,퇴근시간대총승객수,밤시간대총승객수
0,202101,104000001,5001,84,0,0,0,0,36,206,...,127.075646,37.558741,설치,광진구,중곡1동,8564,2002,3720,2230,612
1,202101,104000001,5001,39,59,42,34,133,0,0,...,127.075646,37.558741,설치,광진구,중곡1동,316,0,0,0,316
2,202101,104000001,5001,0,0,0,0,27,81,87,...,127.075646,37.558741,설치,광진구,중곡1동,3633,815,1677,943,198
3,202101,104000001,5001,0,0,0,0,0,75,72,...,127.075646,37.558741,설치,광진구,중곡1동,3310,831,1342,993,144
4,202101,104000001,5001,0,0,0,0,409,503,205,...,127.075646,37.558741,설치,광진구,중곡1동,7291,1921,3201,1051,1118


In [31]:
data_sum = data.drop(morning_cols + afternoon_cols + evening_cols + night_cols, axis=1)

## 2. 정류장 별로 합치기

- 현재 승차데이터는 정류장 별이 아니라 버스 별이기 때문에 정류장 별 승차량을 구할 수 있도록 집계

### 1. 합계 구하기

In [51]:
cols = ['출근시간대총승객수', '낮시간대총승객수', '퇴근시간대총승객수', '밤시간대총승객수', '일일총승객수']
data_merge = data_sum.groupby(by=['사용년월', '표준버스정류장ID'], axis=0, as_index=False)[cols].sum()

In [52]:
# 열이 중복으로 들어가는 것 방지
on_cols = ['사용년월', '표준버스정류장ID']
cols_to_use = data_sum.columns.difference(data_merge.drop(on_cols, axis=1).columns)

In [53]:
data_merge = data_merge.merge(data_sum[cols_to_use], how='inner', on=on_cols, )

In [54]:
data_merge = data_merge.drop_duplicates(subset=on_cols, keep='first', ignore_index=True)

In [57]:
data_merge.to_csv('./data/gwangjin_borading_sum.csv', encoding='utf-8', index=False)